In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
from ipywidgets import *
from bqplot import *
from bqplot.interacts import *

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = pd.read_excel('Equity_IDX_universe_list.xlsx')['ID'].tolist()
# equity_idx = ['SXXP Index', 'SX5E Index', 'IBEX Index', 'DAX Index', 'CAC Index', 'UKX Index', 'SMI Index', 'SPMIB Index', 'OMX Index', 'LCXP Index', 'MCXP Index',
#               'SCXP Index', 'MXEU000V Index', 'MXEU000G Index', 'SPX Index', 'RAY Index', 'CCMP Index', 'NKY Index', 'HSI Index', 'AS51 Index', 'KOSPI Index', 'TWSE Index',
#               'IBOV Index', 'SENSEX Index', 'SHCOMP Index', 'MERVAL Index', 'IPSA Index', 'MEXBOL Index', 'PX Index', 'BUX Index', 'JALSH Index', 'SET Index']

In [2]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(1999,12,31)
end_date = datetime.datetime(2018,12,31)

In [3]:
######################  MAIN  ######################

index_universe = [index_to_replicate] + equity_idx

# Get the the index currency
id_currency = d.crncy()

request = bql.Request(index_universe[0], {'CURRENCY': id_currency})
res = bq.execute(request)
idx_curr = list(res[0].df()['CURRENCY'])

criteria = id_currency == idx_curr[0]
filtered_univ = u.filter(u.list(equity_idx), criteria)

# Compute the correlation matrix

price = d.px_last(start = start_date, end = end_date, per = 'M', fill='prev', currency = 'USD')
price = (100*price)/f.first(price) # Normalize each data series as in Percentage Appreciation but adds an offset of 100 (SAME METHOD WHEN COMPARING TWO SERIES IN GP<GO> AND NORMALIZING)

# price = f.znav(f.pct_diff(f.zscore(price)))/100
# price = 1-f.cumprod(price + 1) #We calculate the z_score of the price and from that z_score we see the cumulative total return to get the correlation

base_price = f.value(price, u.list(index_universe[0]))
correlation = f.corr(base_price, price)
correlation = f.dropna(correlation, REMOVE_ID =True)['value']

request = bql.Request(filtered_univ, {'CORRELATION': correlation}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
res = bq.execute(request)
data_mat = res[0].df().reset_index()

# Sort by Correlation and filter with the index list that have the same currency as the index we want to replicate to avoid currency risks
filtered_data_mat = data_mat.sort_values(by = ['CORRELATION'], ascending = False)

# Get the cumulative performance of de zscore of the price of the filtered assets
filtered_assets = [index_universe[0]] + filtered_data_mat['ID'].tolist()

for i in range(len(filtered_assets)): #We're doing this to get the DF in a way that the columns are the assets and the rows months (in some cases it's difficult to make the monthly alineation, this is why we develop this code)
    request = bql.Request(filtered_assets[i], {filtered_assets[i]: price}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
    res = bq.execute(request)
    
    asset = res[0].df()
    asset = pd.DataFrame({filtered_assets[i]: asset[filtered_assets[i]].tolist(), 'DATE': asset['DATE'].tolist()}, index= list(range(len(asset))))
    
    year_list = pd.DatetimeIndex(asset['DATE']).year.tolist()
    month_list = pd.DatetimeIndex(asset['DATE']).month.tolist()
    date_list = []
    for j in range(len(year_list)):
        date_list += [str(year_list[j]) + '-' + str(month_list[j])]
    asset['aux_date'] = date_list    
    
    if i == 0:
        matrix = asset.copy(deep=True)
    else :
        asset = asset.drop(['DATE'], axis=1)
        matrix = pd.merge(matrix, asset, how='left', on=['aux_date', 'aux_date'])
        

aux_matrix = matrix[filtered_assets]

FIRST WE MEASUERE THE CORRELATIONS BETWEEN THE ASSETS AND THE INDEX AND FILTER FOR THE REST OF THE ANALYSIS BY TAKING THE INDEX/ASSETS WITH A POSITIVE CORRELATION ABOVE 70%

In [4]:
filtered_data_mat = filtered_data_mat[filtered_data_mat['CORRELATION']>=0.7]
filtered_assets = filtered_data_mat['ID'].tolist()+[index_to_replicate]
aux_matrix = aux_matrix[filtered_assets]

In [5]:
x_bar = filtered_data_mat['ID'].tolist()
y_bar = filtered_data_mat['CORRELATION'].tolist()

xs = OrdinalScale()
ys = LinearScale()
sc = ColorScale(mid=0.0)

bar = Bars(x=x_bar, y=y_bar, scales={'x': xs, 'y': ys, 'color': sc}, color=y_bar, padding=0.5)

#To put names it's necessary the attribute names and the scatter plot has it. in order to see only the bars, the opacity of the scatter is going to be 0
# Changing the marker as 
sc_x = LinearScale()
sc_y = LinearScale()

scatt = Scatter(x=x_bar, y=y_bar, names=np.array(y_bar), scales={'x': xs, 'y': ys})

xax = Axis(scale=xs, tick_rotate=-45)
yax = Axis(scale=ys, orientation='vertical', tick_format='0.0%', grid_lines='solid')

plot = Figure(marks=[bar, scatt], axes=[xax, yax])
plot.layout.width = '1500px'
plot

Figure(axes=[Axis(scale=OrdinalScale(), tick_rotate=-45), Axis(orientation='vertical', scale=LinearScale(), ti…

WE SEE THE CONNECTIONS BETWEEN THE ASSETS AND THE INDEX WITH THE DENDROGRAM OF THE CORRELATION MATRIX

In [6]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

def correlDist(corr):
    dist = ((1-corr)/2)**0.5
    return dist

Correlations = aux_matrix.corr(method ='pearson')
Z = linkage(correlDist(Correlations))

max_d = 0.41

plt.figure(figsize=(30, 10), facecolor='w')
dendrogram(Z, labels = filtered_assets, color_threshold = max_d)
plt.show()

In [7]:
correlDist(Correlations)

MXEASC Index  MXMS Index  MXAPJ Index  MXWOSC Index  \
MXEASC Index        0.000000    0.200368     0.201650      0.130943   
MXMS Index          0.200368    0.000000     0.046194      0.229372   
MXAPJ Index         0.201650    0.046194     0.000000      0.242910   
MXWOSC Index        0.130943    0.229372     0.242910      0.000000   
M1EA Index          0.101710    0.176228     0.179232      0.153268   
M1EU Index          0.114170    0.178941     0.175974      0.172985   
RUJ Index           0.159675    0.277128     0.286036      0.074652   
MXEFSC Index        0.251807    0.100663     0.080956      0.287711   
M00IWO$O Index      0.205524    0.274231     0.293505      0.100065   
RMC Index           0.201081    0.282388     0.299827      0.080597   
MXWDIM Index        0.145874    0.230840     0.244104      0.135173   
MID Index           0.217308    0.285986     0.304791      0.091813   
FTAG01 Index        0.284787    0.139013     0.112666      0.338293   
M1WO Index          0.185271    0.265226     0.284791      0.105042   
RTY Index           0.200846    0.288307     0.306083      0.081694   
MXEF Index          0.277634    0.134168     0.107016      0.335166   
FTAW01 Index        0.155144    0.237202     0.249932      0.149039   
SML Index           0.219341    0.300101     0.319047      0.098032   
SPW Index           0.218264    0.299265     0.317232      0.097841   
SPXPV Index         0.204367    0.308399     0.321113      0.099712   
COMP Index          0.217806    0.294510     0.314689      0.114142   
SP5LVI Index        0.239960    0.318455     0.335966      0.127279   
MXWD Index          0.167470    0.250522     0.262888      0.165915   
MXWD000V Index      0.150937    0.256720     0.248919      0.228731   
RLV Index           0.199592    0.327455     0.338738      0.136418   
MXKO Index          0.190420    0.282666     0.296625      0.160575   
M1US000V Index      0.224963    0.323526     0.341079      0.133588   
NDEUSTW Index       0.241130    0.236652     0.267031      0.186577   
MXWO Index          0.193065    0.289169     0.302618      0.175744   
RUO Index           0.258607    0.321258     0.344136      0.153571   
SPXT Index          0.267349    0.342220     0.365216      0.164863   
INDU Index          0.258011    0.335161     0.357372      0.164637   
SVX Index           0.229218    0.353523     0.365121      0.182254   
RAY Index           0.259748    0.345706     0.366387      0.169683   
SPR Index           0.262148    0.348747     0.369528      0.171820   
MXNA Index          0.261552    0.342831     0.363515      0.179450   
RIY Index           0.267678    0.352786     0.373617      0.180294   
IXV Index           0.306443    0.380563     0.402049      0.199597   
MXWOU Index         0.259833    0.296860     0.280702      0.357932   
SPX Index           0.278422    0.364837     0.385696      0.195358   
MXUS Index          0.280236    0.363373     0.384492      0.196958   
MXEA Index          0.272520    0.322344     0.308423      0.369153   
MXUS000G Index      0.319937    0.381191     0.406369      0.230815   
SGX Index           0.326038    0.388595     0.413488      0.236974   
CCMP Index          0.330212    0.383546     0.410148      0.246108   
HFRIFOF Index       0.149233    0.161236     0.162592      0.164364   

                M1EA Index  M1EU Index  RUJ Index  MXEFSC Index  \
MXEASC Index      0.101710    0.114170   0.159675      0.251807   
MXMS Index        0.176228    0.178941   0.277128      0.100663   
MXAPJ Index       0.179232    0.175974   0.286036      0.080956   
MXWOSC Index      0.153268    0.172985   0.074652      0.287711   
M1EA Index        0.000000    0.038096   0.198229      0.238277   
M1EU Index        0.038096    0.000000   0.212962      0.233538   
RUJ Index         0.198229    0.212962   0.000000      0.329684   
MXEFSC Index      0.238277    0.233538   0.329684      0.000000   
M00IWO$O Index    0.208976    0.230086   0.129447      0.335180   
RMC Inde

IN ORDER TO KNOW THE IMPORTANCE (WEIGHT) OF EACH ASSET AS A CHARACTERISTIC OF THE INDEX, WE PERFORM AN RIDGE REGRESSION (SIMILAR TO THE LASSO BUT MORE SPECIFIC WHEN THE UNIVERSE OF CHARACTERISTIC IS LOWER AND THE CHARACTERISTICS, IN THIS CASE THE ASSETS, ARE CORRELATED)

In [8]:
from sklearn import linear_model

#We have to transform the data to give the correcto input for the ridge regression function
X = []
for i in range(len(aux_matrix)):
    X.append(np.array(aux_matrix.iloc[i][filtered_data_mat['ID'].tolist()].tolist()))
X = np.array(X)
y = np.array(aux_matrix[index_universe[0]])
#############

list_alphas = np.arange(0.0001, 0.5, 0.0001)
list_equities = filtered_data_mat['ID'].tolist()

weights = []
for coef_alpha in list_alphas:
    model = linear_model.Ridge(alpha=coef_alpha).fit(X,y)
    weights.append(model.coef_)
    
list_weights= []
for i in range(len(list_equities)):
    aux_weight = []
    for ii in range(len(list_alphas)):
        aux_weight += [weights[ii][i]]
    
    list_weights += [aux_weight]
    
    
########## Plot only the ones that represent big moves and hig coeficcients in the regression

# Calculate the area of the curve in absolute terms
ridge_area = []
for i in range(len(list_equities)):
    ridge_area += [abs(sum(list_weights[i]))]
    
#Normalize and take the values above 0 (the bigger ones)
ridge_area = np.array(ridge_area)

#We will use this data mat when generating the scores to rack the assets
ridge_area_data_mat = pd.DataFrame(data = {'ID': list_equities, 'Ridge': ridge_area})

ridge_area = (ridge_area-np.mean(ridge_area))/np.std(ridge_area)

bool_list = list(ridge_area>=0)

ridge_list_equities = []
ridge_list_weights = []

for i in range(len(list_equities)):
    if bool_list[i] == True:
        ridge_list_equities += [list_equities[i]]
        ridge_list_weights += [list_weights[i]]

In [9]:
x_lg = LogScale()
y_sc = LinearScale()

line = Lines(x=-list_alphas, y=ridge_list_weights, scales={'x': x_lg, 'y': y_sc}, display_legend=True, labels=ridge_list_equities)

ax_x = Axis(scale=x_lg, label='Alpha')
ax_y = Axis(scale=y_sc, orientation='vertical', label = 'Pesos')

plot = Figure(marks=[line], axes=[ax_x, ax_y], legend_location='top-left')
plot.layout.width = '1500px'
plot

Figure(axes=[Axis(label='Alpha', scale=LogScale()), Axis(label='Pesos', orientation='vertical', scale=LinearSc…

FINALLY WE MAKE A PCA TO CONTRAST ALL THIS RESULT IN ORDER TO CONFIRM THE ASSET SELECTION

In [10]:
from sklearn.decomposition import PCA

# The first step is to apply the difference between the series of the assets and the index
aux_matrix2 = aux_matrix.copy(deep=True)
aux_matrix2 = aux_matrix2[list_equities]
for eq in list_equities:
    aux_matrix2[eq] = aux_matrix2[eq] - aux_matrix[index_universe[0]] 

#Then we have to transform the data as in the Ridge regression
X = []
for i in range(len(aux_matrix2)):
    X.append(np.array(aux_matrix2.iloc[i][list_equities].tolist()))
X = np.array(X)
#############

pca = PCA(n_components=len(list_equities))
pca.fit(X)

x_bar = ['PC']*len(list_equities)
for i in range(len(list_equities)):
    x_bar[i] = x_bar[i] + str(i+1)

xs = OrdinalScale()
ys = LinearScale()
sc = ColorScale(mid=0.0)

bar = Bars(x=x_bar, y=pca.explained_variance_ratio_, scales={'x': xs, 'y': ys, 'color': sc}, color=y_bar, padding=0.5)

#To put names it's necessary the attribute names and the scatter plot has it. in order to see only the bars, the opacity of the scatter is going to be 0
# Changing the marker as 
sc_x = OrdinalScale()
sc_y = LinearScale()

scatt = Scatter(x=x_bar, y=pca.explained_variance_ratio_, names=np.array(pca.explained_variance_ratio_), scales={'x': xs, 'y': ys})

xax = Axis(scale=xs)
yax = Axis(scale=ys, orientation='vertical', tick_format='0.0%', grid_lines='solid')

plot = Figure(marks=[bar, scatt], axes=[xax, yax], title = 'Varianza explicada')
plot.layout.width = '1500px'
plot

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='0.0%')…

In [11]:
explained_components = []
for i in range(len(list_equities)):
    
    #It must invert the weights of each components because we're computing the spread between an asset and the index to replicate
    aux_inv_PC = 1/pca.components_[i]
    
    #Some values are close to inf, so we replace them with the next maximum of the array
    aux_inv_PC[aux_inv_PC == -np.inf] = 0
    aux_inv_PC[aux_inv_PC == np.inf] = 0
    aux_inv_PC[aux_inv_PC == 0] = max(aux_inv_PC)
    
    explained_components += [aux_inv_PC*pca.explained_variance_ratio_[i]]
    
total_explained_components = []    
for i in range(len(list_equities)):
    sum_cont = 0
    for ii in range(len(list_equities)):
        sum_cont += abs(explained_components[ii][i])
        
    total_explained_components += [sum_cont]

pca_mat = pd.DataFrame(data = {'ID': list_equities,'Explained PCA': total_explained_components})
pca_mat = pca_mat.sort_values(by=['Explained PCA'], ascending=False)
    
#Plot the significance of each asset

xs = OrdinalScale()
ys = LinearScale()
sc = ColorScale(mid=0.0)

bar = Bars(x=pca_mat['ID'].tolist(), y=pca_mat['Explained PCA'].tolist(), scales={'x': xs, 'y': ys, 'color': sc}, color=pca_mat['Explained PCA'].tolist(), padding=0.5)

#To put names it's necessary the attribute names and the scatter plot has it. in order to see only the bars, the opacity of the scatter is going to be 0
# Changing the marker as 
sc_x = OrdinalScale()
sc_y = LinearScale()

scatt = Scatter(x=pca_mat['ID'].tolist(), y=pca_mat['Explained PCA'].tolist(), names=np.array(pca_mat['Explained PCA'].tolist()), scales={'x': xs, 'y': ys})

xax = Axis(scale=xs, tick_rotate=-45)
yax = Axis(scale=ys, orientation='vertical', tick_format='0.0', grid_lines='solid')

plot = Figure(marks=[bar, scatt], axes=[xax, yax], title = 'Valor total de SUMA(Varianza explicada * peso CP) de cada variable')
plot.layout.width = '1500px'
plot

Figure(axes=[Axis(scale=OrdinalScale(), tick_rotate=-45), Axis(orientation='vertical', scale=LinearScale(), ti…

In [12]:
#Normalize and take the values above 0 (the bigger ones)
filter_pca_explained = np.array(pca_mat['Explained PCA'].tolist())
filter_pca_explained = (filter_pca_explained-np.mean(filter_pca_explained))/np.std(filter_pca_explained)

filtered_pca_mat = pca_mat[list(filter_pca_explained>=0)]

orig_pca_mat = pd.DataFrame(data = {'ID': list_equities,'PC1': pca.components_[0], 'PC2': pca.components_[1]})
filtered_pca_mat = pd.merge(filtered_pca_mat, orig_pca_mat, how = 'left', on=['ID'])

sc_x = LinearScale()
sc_y = LinearScale()

scatt = Scatter(x=filtered_pca_mat['PC1'].tolist(), y=filtered_pca_mat['PC2'].tolist(), names=filtered_pca_mat['ID'].tolist(), scales={'x': sc_x, 'y': sc_y}, apply_clip=False)

xax = Axis(scale=sc_x, label = 'PC1', )
yax = Axis(scale=sc_y, orientation='vertical', grid_lines='solid', label = 'PC2')

plot = Figure(marks=[scatt], axes=[xax, yax])
plot.layout.width = '1500px'
plot

Figure(axes=[Axis(label='PC1', scale=LinearScale()), Axis(label='PC2', orientation='vertical', scale=LinearSca…

CONSIDERING ALL THE INPUTS WE GET FROM THE STATISTICAL ANALYSIS, WE GENERATE A SCORE FOR EACH PART

In [13]:
from scipy.cluster.hierarchy import fcluster

score_stats_mat = filtered_data_mat.copy(deep = True)

#We've observed that the index HFRIFOF it should be in the cluster defined by the distance 0.41
#This is why we score the rest of the parts in quintiles
clusters = fcluster(Z, max_d, criterion='distance')
cluster_data_mat = pd.DataFrame(data = {'ID':filtered_assets, 'Cluster': clusters})
# The main cluster is number 3, so we're top ranking in the quintile. Next to this cluster is 2, the cluster number 4 becomes the 3rd most important,
#number 5 is the 4th and cluster number 1 the fifth
#We compute the transformation

cluster_data_mat = cluster_data_mat.replace({'Cluster':3}, 'a')
cluster_data_mat = cluster_data_mat.replace({'Cluster':2}, 'b')
cluster_data_mat = cluster_data_mat.replace({'Cluster':4}, 'c')
cluster_data_mat = cluster_data_mat.replace({'Cluster':5}, 'd')
cluster_data_mat = cluster_data_mat.replace({'Cluster':1}, 'e')

cluster_data_mat = cluster_data_mat.replace({'Cluster':'a'}, 1)
cluster_data_mat = cluster_data_mat.replace({'Cluster':'b'}, 2)
cluster_data_mat = cluster_data_mat.replace({'Cluster':'c'}, 3)
cluster_data_mat = cluster_data_mat.replace({'Cluster':'d'}, 4)
cluster_data_mat = cluster_data_mat.replace({'Cluster':'e'}, 5)

score_stats_mat = pd.merge(score_stats_mat, cluster_data_mat, how = 'left', on=['ID'])

#Add the info obtained in the Ridge Regression
score_stats_mat = pd.merge(score_stats_mat, ridge_area_data_mat, how = 'left', on=['ID'])

#Add the info obtained in the PCA
score_stats_mat = pd.merge(score_stats_mat, pca_mat, how = 'left', on=['ID'])

#Get teh quintiles for each part
score_stats_mat['Score_Correlation'] = pd.qcut(score_stats_mat['CORRELATION'], 5, labels=np.arange(5, 0, -1))
score_stats_mat['Score_Cluster'] = score_stats_mat['Cluster']
score_stats_mat['Score_Ridge'] = pd.qcut(score_stats_mat['Ridge'], 5, labels=np.arange(5, 0, -1))
score_stats_mat['Score_PCA'] = pd.qcut(score_stats_mat['Explained PCA'], 5, labels=np.arange(5, 0, -1))
score_stats_mat['TOTAL_Score'] = score_stats_mat[['Score_Correlation', 'Score_Cluster', 'Score_Ridge', 'Score_PCA']].sum(axis=1)
score_stats_mat = score_stats_mat.sort_values(by = ['TOTAL_Score'])
score_stats_mat

ID  CORRELATION  Cluster         Ridge  Explained PCA  \
4       M1EA Index     0.944812        1   7179.973537      55.181199   
10    MXWDIM Index     0.900183        3   9990.672846      44.144600   
16    FTAW01 Index     0.891795        3  11822.038394      50.504054   
22      MXWD Index     0.874726        3  12692.675396     101.572651   
3     MXWOSC Index     0.945969        3   1221.219394      32.211610   
5       M1EU Index     0.944201        1   1213.781699      23.299226   
23  MXWD000V Index     0.874670        1   2114.996432     106.588727   
14       RTY Index     0.893208        3   8919.462863      23.939350   
25      MXKO Index     0.857308        3   8237.099736      44.606286   
35      MXNA Index     0.802894        3   9036.495145      17.645453   
2      MXAPJ Index     0.947128        2    896.997511      14.246545   
28      MXWO Index     0.846327        3   8858.137956      89.209601   
8   M00IWO$O Index     0.913811        3    125.113838      41.209004   
9        RMC Index     0.904167        3   3835.815803      16.868114   
12    FTAG01 Index     0.894267        2   2233.528987      13.398142   
32       SVX Index     0.812373        3   3152.597850      20.329165   
0     MXEASC Index     0.955459        1   1129.478755       7.894992   
1       MXMS Index     0.948006        2   1036.993866      11.280967   
13      M1WO Index     0.893826        3   2312.221272      16.199493   
18       SPW Index     0.888649        3   1229.295121      15.621011   
33       RAY Index     0.809863        3  14573.844387      12.937461   
6        RUJ Index     0.925281        3   2195.581957       5.324186   
40      MXUS Index     0.779064        3   4646.231936      16.985643   
39       SPX Index     0.780313        3   5256.043004      16.921571   
34       SPR Index     0.807555        3  10784.301040      12.401026   
36       RIY Index     0.798524        3  11923.315784      13.514849   
15      MXEF Index     0.892925        2    366.656895      15.359685   
24       RLV Index     0.861015        3    898.133237      12.963571   
11       MID Index     0.899483        3   1351.474203       6.008476   
38     MXWOU Index     0.780371        5  13700.729238      24.010031   
7     MXEFSC Index     0.923097        2     65.203324       7.317999   
17       SML Index     0.890606        3   1113.582612       5.527924   
42  MXUS000G Index     0.741059        3   2400.676677      12.648882   
29       RUO Index     0.831591        3   4655.919393       7.603217   
26  M1US000V Index     0.854481        3   1129.422300       9.373685   
41      MXEA Index     0.751920        5   1779.061744      22.603671   
44      CCMP Index     0.725446        3   1939.304769      12.078992   
27   NDEUSTW Index     0.848878        4    470.513450      20.556484   
43       SGX Index     0.731403        3    658.100354      13.662069   
20      COMP Index     0.881952        3    126.070824       9.440218   
19     SPXPV Index     0.887472        3    300.987287       9.890204   
31      INDU Index     0.821215        3    667.453158      11.958729   
21    SP5LVI Index     0.881406        3    202.907763       7.889176   
30      SPXT Index     0.822027        3    114.652413       7.212315   
37       IXV Index     0.797501        3     57.760001       5.905598   

   Score_Correlation  Score_Cluster Score_Ridge Score_PCA  TOTAL_Score  
4                  1              1           2         1          5.0  
10                 2              3           1         1          7.0  
16                 2              3           1         1          7.0  
22                 3              3           1         1          8.0  
3                  1              3           3         1          8.0  
5                  1              1           4         2          8.0  
23                 3              1           3         1          8.0  
14                 2              3           1         2          8.0  


TO CONSIDER THE AVAILABILITY OF MAKING THIS ANALYSIS REAL, WE SHOULD CONTRATS THE SCORES WITH THE MARKET CAP OF EACH INDEX

In [14]:
market_cap = d.CUR_MKT_CAP()['value']

request = bql.Request(u.list(score_stats_mat['ID'].tolist()), {'Market Cap': market_cap}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
res = bq.execute(request)
score_stats_mat = pd.merge(score_stats_mat, res[0].df().reset_index(), how = 'left', on=['ID'])

sc_x = LinearScale()
sc_y = LinearScale()

scatt = Scatter(x=score_stats_mat['TOTAL_Score'].tolist(), y=score_stats_mat['Market Cap'].tolist(), names=score_stats_mat['ID'].tolist(), scales={'x': sc_x, 'y': sc_y}, apply_clip=False)

xax = Axis(scale=sc_x, label = 'TOTAL_Score', )
yax = Axis(scale=sc_y, orientation='vertical', grid_lines='solid', label = 'Market Cap')

plot = Figure(marks=[scatt], axes=[xax, yax])
plot.layout.width = '1500px'
plot.layout.height = '700px'
plot

Figure(axes=[Axis(label='TOTAL_Score', scale=LinearScale()), Axis(label='Market Cap', orientation='vertical', …